## Import

In [1]:
import sys
sys.path.append(r'\\Pund\Stab$\guest801981\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')
sys.path.append(r'C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')

In [2]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
# Ignore the ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
import os
import dill
import random
import inspect
import numpy as np
import pandas as pd
import pickle as pkl
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Point
from log import print_INFO_message_timestamp, print_INFO_message
from facility_location import (FacilityLocation,
                               StochasticFacilityLocation)
from retrieve_global_parameters import retrieve_adj_matrix_path

[    0.00] Initializing mpi-sppy


## Data loading

In [4]:
root_path = r"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\07_model_output"
root_path = r'\\Pund\Stab$\guest801981\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\07_model_output'

In [8]:
n_facilities = [1,2,3]
times = ["morning", "midday", "afternoon"]

In [9]:
stochastic_solutions = {}

for n in n_facilities:
    stochastic_solutions[n] = StochasticFacilityLocation.load(os.path.join(root_path, 
                                                                          rf"{n}_locations\stochastic_solution\lshape_solution.pkl"))

In [12]:
deterministic_scenarios_solution = {}
for n in n_facilities:
    for time in times:
        deterministic_scenarios_solution[(n, time)] = FacilityLocation.load(os.path.join(root_path, 
                                                                                         rf"{n}_locations\deterministic_exact_solutions\light_exact_solution_{time}.pkl"))

In [13]:
deterministic_free_flow_solution = {}

for n in n_facilities:
    deterministic_free_flow_solution[n] = FacilityLocation.load(os.path.join(root_path, 
                                                                             rf"{n}_locations\deterministic_exact_solutions\light_exact_solution_all_day_free_flow.pkl"))

## Solution analysis

In [15]:
print_INFO_message_timestamp("Objective value for the Stochastic solution")
for (n, fl_sto), fl_det in zip(stochastic_solutions.items(), deterministic_free_flow_solution.values()):
    print_INFO_message(f"locations: {n}    solution: \n"+
                       f"                                             deterministic: {round(fl_det.solution_value/60,3)} minutes\n"+
                       f"                                             stochastic: {round(fl_sto.solution_value/60,3)} minutes \n")

[09/19/23 15:14:22] INFO     Objective value for the Stochastic solution
                    INFO     locations: 1    solution: 
                                             deterministic: 27.607 minutes
                                             stochastic: 28.498 minutes 

                    INFO     locations: 2    solution: 
                                             deterministic: 16.693 minutes
                                             stochastic: 17.912 minutes 

                    INFO     locations: 3    solution: 
                                             deterministic: 16.318 minutes
                                             stochastic: 16.629 minutes 



In [22]:
n = 3
print_INFO_message_timestamp("Objective value for the Stochastic solution")
for time in times:
    fl_det = deterministic_scenarios_solution[(n, time)]
    fl_sto = stochastic_solutions[n]
    print_INFO_message(f"locations: {n}    solution: \n"+
                       f"                                             deterministic ({time}): {fl_det.locations_coordinates}\n"+
                       f"                                             stochastic: {fl_sto.locations_coordinates}\n")

[09/19/23 15:18:42] INFO     Objective value for the Stochastic solution
                    INFO     locations: 3    solution: 
                                             deterministic (morning): [geometry    POINT (5.45545 60.37507)
Name: 8228, dtype: geometry, geometry    POINT (5.26533 60.33305)
Name: 2560, dtype: geometry, geometry    POINT (5.30379 60.43890)
Name: 4342, dtype: geometry]
                                             stochastic: [geometry    POINT (5.32593 60.46721)
Name: 4979, dtype: geometry, geometry    POINT (5.26533 60.33305)
Name: 2560, dtype: geometry, geometry    POINT (5.46010 60.42187)
Name: 5313, dtype: geometry]

                    INFO     locations: 3    solution: 
                                             deterministic (midday): [geometry    POINT (5.46684 60.41821)
Name: 9795, dtype: geometry, geometry    POINT (5.32593 60.46721)
Name: 4971, dtype: geometry, geometry    POINT (5.26533 60.33305)
Name: 2560, dtype: geometry]
                     

In [9]:
for k, v in stochastic_solutions[1].__dict__.items():
    print(k)

coordinates
candidate_coordinates
n_of_locations_to_choose
n_of_demand_points
model
scenarios_names
scenarios_probabilities
solution_value
locations_index
locations_coordinates
first_stage_solution
second_stage_solution
solver_status
computation_time


In [10]:
idx = pd.Series([k if stochastic_solutions[3].first_stage_solution[k] != 0 else None 
                 for k in stochastic_solutions[3].first_stage_solution.keys()]).dropna().values
idx

array([10., 21., 35.])

In [22]:
n = 3

In [23]:
fig = go.Figure()

fl_det = deterministic_free_flow_solution[n]
fl_sto = stochastic_solutions[n]

idx = pd.Series([k if stochastic_solutions[n].first_stage_solution[k] != 0 else None 
                 for k in stochastic_solutions[n].first_stage_solution.keys()]).dropna().values

stochastic_locations_coordinates = stochastic_solutions[n].candidate_coordinates.iloc[[int(i) for i in idx]]

fig.add_trace(go.Scattermapbox(
    lat=fl_det.coordinates.geometry.y,
    lon=fl_det.coordinates.geometry.x,
    mode='markers',
    marker=dict(
        color=["grey"]*fl_det.coordinates.shape[0],
        size=4.5,
    ),
    hovertemplate='<extra></extra>',
    showlegend=False,
))

fig.add_trace(go.Scattermapbox(
    lat=[p.geometry.y for p in fl_det.locations_coordinates],
    lon=[p.geometry.x for p in fl_det.locations_coordinates],
    mode='markers',
    marker=dict(
        color=["red"]*len(fl_det.locations_coordinates),
        size=6,
    ),
    hovertemplate='<br>Latitude: %{lat}<br>Longitude: %{lon}<extra></extra>',
    name="deterministic",
    showlegend=True,
))
    
fig.add_trace(go.Scattermapbox(
    lat=[p.y for p in stochastic_locations_coordinates.geometry],
    lon=[p.x for p in stochastic_locations_coordinates.geometry],
    mode='markers',
    marker=dict(
        color=["blue"]*len(stochastic_locations_coordinates),
        size=6,
    ),
    hovertemplate=f'<br>solution value: <extra></extra>',
    name="stochastic",
    showlegend=True,
))

fig.update_layout(title="<b>deterministic vs stochastic solution<b>",
                  mapbox=dict(
                    style="open-street-map",
                    center=dict(lat=fl_det.coordinates.geometry.y.mean(), lon=fl_det.coordinates.geometry.x.mean()),
                    zoom=9
                    ),
                  title_pad_l=260,
                  height=700,
                  width=1000,
                  xaxis_title="time of the day",)

fig.show()

: 